Solver

In [22]:
using JuMP
using Gurobi
using CSV
using DataFrames
using Plots

Data Importation

In [23]:
budgets_demand = CSV.File("./data/budgets_demand.csv",header=0) |> Tables.matrix;
impression_data = CSV.File("./data/impression_per_age.csv",header=0) |> Tables.matrix; # plaform, age group, impressions rate
company_rcs_data = CSV.File("./data/company_rcs.csv",header=0) |> Tables.matrix; # company, rps
platform_variables_data = CSV.File("./data/platform_variables.csv",header=0) |> Tables.matrix; # platform, variable, value


Clean Data

In [24]:
# define dictionaries to use

# B: budget
budget_dict = Dict() # dict of (company) -> budget; 50 companies
for i in 2:size(budgets_demand,1)
    budget_dict[budgets_demand[i, 1]] = parse(Int, budgets_demand[i, 2])
end

# R_a: total reach demanded for each age group
demand_dict = Dict() # dict of (company, age group) -> demand; 50 companies, 4 age groups
age_groups = ["18-24", "25-34", "35-44", "45-60"]
for i in 2:size(budgets_demand,1)
    for j in 3:size(budgets_demand,2)
        demand_dict[(budgets_demand[i, 1], budgets_demand[1,j])] = parse(Int, budgets_demand[i,j])
    end
end

# impression rate per age group (a_ap*i_p)
impressions_per_age = Dict() # dict of (platform, age group) -> impression rate
for i in 2:size(impression_data,1)
    str_value = string(impression_data[i, 3])  # Convert to regular String
    impressions_per_age[(impression_data[i, 1], impression_data[i, 2])] = parse(Float64, str_value)  # Parse to Int
end


# company revenue per conversion
company_rcs = Dict() # dict of company -> r_c
for i in 1:size(company_rcs_data,2)
    # row 1 is company name, row 2 is r_c
    company_rcs[company_rcs_data[1, i]] = parse(Float64, company_rcs_data[2, i])
end

# platform variables
# constant for all companies need for each platform, 
    # w_p: click_rate
    # i_p: impression_rate
    # f_p: average_impression_rate
    # c_p: conversion rate
    # s_p: cost per click 

platform_variables = Dict() # dict of (platform, variable) -> value
for i in 2:size(platform_variables_data, 2)
    for j in 2:size(platform_variables_data, 1)
        platform_variables[platform_variables_data[1, i], platform_variables_data[j, 1]] = parse(Float64, platform_variables_data[j, i])
    end
end



In [25]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


Solver

In [26]:
platforms_l = ["Facebook", "Instagram", "Twitter"] # keept this order
companies_l = collect(keys(budget_dict)) # keept this order
age_groups = ["18-24", "25-34", "35-44", "45-60"]


# model for each company
optimized_solutions= Dict() # dict of company -> {t -> [t_p], optimal_value -> value}

for i in 1:length(companies_l)
    company = companies_l[i]
    println("Optimizing for company: ", company)
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))

    ### decision variable ###
    # hours of advertisement allocated to platform p by company c: t_p
    @variable(model, t[1:length(platforms_l)] >= 0);

    ### constraints ###
    # reach for certain age group must reach a min
    @constraint(model, demand_constraints[a in 1:length(age_groups)], sum(((impressions_per_age[platforms_l[p], age_groups[a]] * t[p])/platform_variables[platforms_l[p], "f_p"] for p in 1:length(platforms_l))) >= demand_dict[company, age_groups[a]]);

    # budget constraint
    # @constraint(model, sum(platform_variables[platforms_l[p], "s_p"] * (platform_variables[platforms_l[p], "w_p"]*(platform_variables[platforms_l[p], "i_p"]*t[p])/platform_variables[platforms_l[p], "f_p"]) for p in 1:length(platforms_l)) <= budget_dict[company]);


    ### objective function ### 
    @objective(model, Min,
    sum((
        sum(platform_variables[platforms_l[p], "s_p"] * (platform_variables[platforms_l[p], "w_p"]*(platform_variables[platforms_l[p], "i_p"]*t[p])/platform_variables[platforms_l[p], "f_p"]) for p in 1:length(platforms_l)))
    ));

    optimize!(model)
    optimized_solutions[company] = Dict()


    dual_values = [dual(demand_constraints[a]) for a in 1:length(age_groups)]

    try
        optimized_solutions[company]["optimal_budget"] = objective_value(model)
        optimized_solutions[company]["t"] = value.(t)
        optimized_solutions[company]["dual_values"] = dual_values
    catch e
        optimized_solutions[company]["optimal_budget"] = -1 # no solution
        optimized_solutions[company]["t"] = -1
    end
end



Optimizing for company: Silk & Style
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 3 columns and 12 nonzeros
Model fingerprint: 0x94f3c9af
Coefficient statistics:
  Matrix range     [6e-01, 3e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.702000e+03   0.000000e+00      0s
       3    1.4200969e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.420096873e+03

User-callback calls 44, time in user-callback 0.00 sec
Optimizing for company: Vogue Visions
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)


In [27]:
for (company, solution) in optimized_solutions
    println("Company: ", company)
    println("Proposed budget: ", solution["optimal_budget"])
    println("Optimal t: ", solution["t"])
    println("Dual values: ", solution["dual_values"])
    println()
end

Company: Silk & Style
Proposed budget: 1420.0968729971398
Optimal t: [101.70753826743362, 69.7424839564212, 434.3231083638419]
Dual values: [1.3310067988139078, 0.5333618956639448, 0.0, 0.36922177377979787]

Company: Vogue Visions
Proposed budget: 1227.258491751657
Optimal t: [0.0, 174.47562427981862, 335.7126606225467]
Dual values: [2.818622454011731, 0.0, 0.08039597348061467, 0.0]

Company: Hearth Harmony
Proposed budget: 1367.9654333968488
Optimal t: [0.0, 517.9866195764649, 0.0]
Dual values: [2.941861147089998, 0.0, 0.0, 0.0]

Company: Silicon Saga
Proposed budget: 1853.9465729011915
Optimal t: [0.0, 442.65064352741, 299.9975193002428]
Dual values: [2.8171893853441636, 0.11538566993997924, 0.0, 0.0]

Company: Furnish Fine
Proposed budget: 1518.456054800027
Optimal t: [12.900600076853307, 9.105941637057116, 640.9675356830581]
Dual values: [1.3310067988139078, 0.5333618956639448, 0.0, 0.36922177377979787]

Company: Gastronomy Guild
Proposed budget: 2500.581975026498
Optimal t: [0.0, 

Baselines - Random

In [28]:
platforms_l = ["Facebook", "Instagram", "Twitter"] # keept this order
companies_l = collect(keys(budget_dict)) # keept this order
age_groups = ["18-24", "25-34", "35-44", "45-60"]

baseline_random = Dict() # dict of company -> {t -> [t_p], optimal_value -> value}

for i in 1:length(companies_l)
    company = companies_l[i]

    # for each age group, pick a random platform
    t = [0.0, 0.0, 0.0]
    for a in 1:length(age_groups)
        # Calculate how much t is needed to meet that age gr demand
        platform_i = rand(1:3)
        platform = platforms_l[platform_i]

        t_needed = demand_dict[company, age_groups[a]] * platform_variables[platform, "f_p"] / impressions_per_age[platform, age_groups[a]]
        t[platform_i] += t_needed
    end

    baseline_random[company] = Dict()
    baseline_random[company]["t"] = t
    baseline_random[company]["random_budget"] = sum(platform_variables[platforms_l[p], "s_p"] * (platform_variables[platforms_l[p], "w_p"]*(platform_variables[platforms_l[p], "i_p"]*t[p])/platform_variables[platforms_l[p], "f_p"]) for p in 1:length(platforms_l))
end


In [29]:
for (company, solution) in baseline_random
    println("Company: ", company)
    println("Optimal budget: ", solution["random_budget"])
    println("Optimal t: ", solution["t"])
    println()
end

Company: Silk & Style
Optimal budget: 4251.331611422518
Optimal t: [414.7921413373871, 501.2773737836757, 845.8567299611377]

Company: Vogue Visions
Optimal budget: 4662.5295104943525
Optimal t: [0.0, 956.7636133569202, 935.4574257020481]

Company: Hearth Harmony
Optimal budget: 2662.950177034203
Optimal t: [503.79725469309267, 517.9866195764649, 37.1966585178676]

Company: Silicon Saga
Optimal budget: 5027.629379613858
Optimal t: [0.0, 679.6760730534878, 1415.8771211175329]

Company: Furnish Fine
Optimal budget: 5027.355721171428
Optimal t: [375.6687446392359, 992.8968703388114, 658.2480105572641]

Company: Gastronomy Guild
Optimal budget: 4955.4336297748
Optimal t: [629.1283439345392, 322.81604185308504, 1135.1927318009134]

Company: Dwell Delight
Optimal budget: 5361.770866204315
Optimal t: [303.04634987542835, 0.0, 2029.6079556668153]

Company: Cozy Corners
Optimal budget: 3678.8499852581695
Optimal t: [342.3419150765511, 935.0272019466399, 169.6111493396659]

Company: Fitness Fron

Baseline - Greedy

In [30]:
platforms_l = ["Facebook", "Instagram", "Twitter"] # keept this order
companies_l = collect(keys(budget_dict)) # keept this order
age_groups = ["18-24", "25-34", "35-44", "45-60"]

baseline_greedy = Dict() # dict of company -> {t -> [t_p], optimal_value -> value}

for i in 1:length(companies_l)
    company = companies_l[i]

    # for each age group, pick a random platform
    t = [0.0, 0.0, 0.0]
    for a in 1:length(age_groups)
        # Calculate how much t is needed to meet that age gr demand
        all_impressions = [impressions_per_age[platforms_l[p], age_groups[a]] for p in 1:length(platforms_l)]
        platform_i = argmax(all_impressions)
        platform = platforms_l[platform_i]

        t_needed = demand_dict[company, age_groups[a]] * platform_variables[platform, "f_p"] / impressions_per_age[platform, age_groups[a]]
        t[platform_i] += t_needed
    end

    baseline_greedy[company] = Dict()
    baseline_greedy[company]["t"] = t
    baseline_greedy[company]["greedy_budget"] = sum(platform_variables[platforms_l[p], "s_p"] * (platform_variables[platforms_l[p], "w_p"]*(platform_variables[platforms_l[p], "i_p"]*t[p])/platform_variables[platforms_l[p], "f_p"]) for p in 1:length(platforms_l))
end


In [31]:
for (company, solution) in baseline_greedy
    println("Company: ", company)
    println("Optimal budget: ", solution["greedy_budget"])
    println("Optimal t: ", solution["t"])
    println()
end

Company: Silk & Style
Optimal budget: 4290.258835989907
Optimal t: [0.0, 893.2635006143314, 845.8567299611377]

Company: Vogue Visions
Optimal budget: 3796.1886621810054
Optimal t: [0.0, 712.900189863994, 838.0851569646285]

Company: Hearth Harmony
Optimal budget: 2372.378060999512
Optimal t: [0.0, 714.3772348445622, 212.75870995666682]

Company: Silicon Saga
Optimal budget: 4272.701857188223
Optimal t: [0.0, 788.2891702056545, 959.5913401918096]

Company: Furnish Fine
Optimal budget: 4543.916162208385
Optimal t: [0.0, 907.5328654046615, 940.4512019946974]

Company: Gastronomy Guild
Optimal budget: 4599.141693697969
Optimal t: [0.0, 1248.599117843383, 570.1261988656901]

Company: Dwell Delight
Optimal budget: 4510.703310233743
Optimal t: [0.0, 1104.6916451207999, 697.8495030155946]

Company: Cozy Corners
Optimal budget: 2763.2960587077932
Optimal t: [0.0, 421.74567058139496, 722.4664089863447]

Company: Fitness Front
Optimal budget: 4134.3766256908475
Optimal t: [0.0, 1174.063305768502